In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('train_clean.csv')
numeric_cols = df.select_dtypes(include=['number']).columns
categ_cols = df.select_dtypes(include=['object']).columns

In [18]:
#PREPROCESAMIENTO
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
#Escalamiento para las variables numericas
#Dummies para las variables categoricas
preprocessing = ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_cols[:-1]),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categ_cols)
    ])

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
modelo = GradientBoostingRegressor(learning_rate=0.2,n_estimators=200) #0.018
pipeline = Pipeline(steps=[('preprocessor', preprocessing),
                           ('model', modelo)])

In [32]:
from sklearn.model_selection import GridSearchCV
# Definir los hiperparámetros que deseas ajustar y los valores posibles
param_grid = {
    'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],  # Los tipos de perdida'squared_error’, ‘absolute_error’, ‘huber’, ‘quantile’},  # Número de estimadores (árboles)
    'learning_rate': np.linspace(0.01, 0.1, 1),  # Tasa de aprendizaje
    'n_estimators': [50, 100, 200, 300, 500],  # NÃ®mero de estimadores,  # Profundidad máxima del árbol
    'subsample': np.linspace(0.1, 1, 1),  # Proporción de muestras en cada nodo,  # Mínimo de muestras para dividir un nodo
    'criterion': ['friedman_mse', 'squared_error']  # Mínimo de muestras en una hoja
}

# Realizar una búsqueda de hiperparámetros utilizando Grid Search Cross-Validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)  # X_train e y_train son tus datos de entrenamiento

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)


ValueError: Invalid parameter 'criterion' for estimator Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBat...
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('model',
                 GradientBoostingRegressor(learning_rate=0.2,
                          

In [20]:
X = df.drop('SalePrice',axis=1)
y = df['SalePrice']

In [21]:
pipeline.fit(X, y)
y_pred = pipeline.predict(X)
from sklearn.metrics import mean_squared_error
import math
math.sqrt(mean_squared_error(np.log(y), np.log(y_pred)))

0.04632793120591926

In [29]:
#version = 1

Prueba: Leer test, llenar NA y hacer predicciones. Exportar a csv

In [30]:
test = pd.read_csv('test.csv')
X_ = test.drop(['Id'],axis=1)
y_ = test['Id']
media = X_[['YearBuilt', 'YearRemodAdd','GarageYrBlt','YrSold']].mean()
X_[['YearBuilt', 'YearRemodAdd','GarageYrBlt','YrSold']].fillna(media)
X_.fillna(0,inplace=True)
results = pipeline.predict(X_)

csv =pd.DataFrame({'Id':test['Id'], 'SalePrice': results})
csv.to_csv(f'predicciones/test_results_{version}.csv',index=False)
version += 1